# TTP

In [12]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

In [13]:
%pip install openpyxl

df = pd.read_excel('../data/processed/DB People.xlsx', sheet_name='DB')

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.1.2 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [14]:
#Visualizar que la base de datos queda bien cargada
df.head()

,GERENCIA,Clave,Nombre,Organización,Departamento,Fecha,Hora,Terminal,Tipo,Tipo Acceso
0,Calidad,38600035,CLARA LIGEIA RUIZ HERNANDEZ,Bavaria S.A.,QUALITY MANAGEMENT,2025-06-03,07:36:00,Tibasosa Der Entrada Contratistas,Asistencia,Entrada
1,Calidad,38600035,CLARA LIGEIA RUIZ HERNANDEZ,Bavaria S.A.,QUALITY MANAGEMENT,2025-06-03,17:17:00,Tibasosa Izq Salida Funcionarios,Asistencia,Salida
2,Calidad,38600035,CLARA LIGEIA RUIZ HERNANDEZ,Bavaria S.A.,QUALITY MANAGEMENT,2025-06-04,05:02:00,Tibasosa Der Entrada Contratistas,Asistencia,Entrada
3,Calidad,38600035,CLARA LIGEIA RUIZ HERNANDEZ,Bavaria S.A.,QUALITY MANAGEMENT,2025-06-04,06:02:00,Tibasosa Izq Salida Funcionarios,Asistencia,Salida
4,Calidad,38600035,CLARA LIGEIA RUIZ HERNANDEZ,Bavaria S.A.,QUALITY MANAGEMENT,2025-06-04,09:58:00,Tibasosa Der Entrada Contratistas,Asistencia,Entrada


In [19]:
# Se pasala columna Hora a formato datetime
#df['Hora'] = pd.to_datetime(df['Hora'])

# Define los horarios de entrada posibles
horarios = [pd.to_datetime('08:00').time(), pd.to_datetime('16:00').time(), pd.to_datetime('00:00').time()]

def calcular_entrada_real(row):
    if row['Tipo Acceso'].lower() != 'entrada':
        return np.nan
    # Convertir 'Hora' a datetime si es string, luego obtener la hora y la fecha
    if isinstance(row['Hora'], str):
        hora_dt = pd.to_datetime(row['Hora'])
    elif isinstance(row['Hora'], pd.Timestamp):
        hora_dt = row['Hora']
    else:
        # Si es datetime.time, combinar con la fecha
        hora_dt = pd.Timestamp.combine(row['Fecha'], row['Hora'])
    for h in horarios:
        h_dt = pd.Timestamp.combine(hora_dt.date(), h)
        if (h_dt - pd.Timedelta(hours=7)) <= hora_dt <= h_dt:
            return h.strftime('%H:%M')
    return np.nan

df['ENTRADA REAL'] = df.apply(calcular_entrada_real, axis=1)

# Visualiza el resultado
df


,GERENCIA,Clave,Nombre,Organización,Departamento,Fecha,Hora,Terminal,Tipo,Tipo Acceso,ENTRADA REAL
0,Calidad,38600035,CLARA LIGEIA RUIZ HERNANDEZ,Bavaria S.A.,QUALITY MANAGEMENT,2025-06-03,07:36:00,Tibasosa Der Entrada Contratistas,Asistencia,Entrada,08:00
1,Calidad,38600035,CLARA LIGEIA RUIZ HERNANDEZ,Bavaria S.A.,QUALITY MANAGEMENT,2025-06-03,17:17:00,Tibasosa Izq Salida Funcionarios,Asistencia,Salida,NaN
2,Calidad,38600035,CLARA LIGEIA RUIZ HERNANDEZ,Bavaria S.A.,QUALITY MANAGEMENT,2025-06-04,05:02:00,Tibasosa Der Entrada Contratistas,Asistencia,Entrada,08:00
3,Calidad,38600035,CLARA LIGEIA RUIZ HERNANDEZ,Bavaria S.A.,QUALITY MANAGEMENT,2025-06-04,06:02:00,Tibasosa Izq Salida Funcionarios,Asistencia,Salida,NaN
4,Calidad,38600035,CLARA LIGEIA RUIZ HERNANDEZ,Bavaria S.A.,QUALITY MANAGEMENT,2025-06-04,09:58:00,Tibasosa Der Entrada Contratistas,Asistencia,Entrada,16:00
...,...,...,...,...,...,...,...,...,...,...,...
4747,People,38613828,MARIANA SANCHEZ QUIROGA,Bavaria & CIA S.C.A,BP SUPPLY MANAGER,2025-06-11,17:18:00,Tibasosa Izq Salida Funcionarios,Asistencia,Salida,NaN
4748,People,38613828,MARIANA SANCHEZ QUIROGA,Bavaria & CIA S.C.A,BP SUPPLY MANAGER,2025-06-12,07:40:00,Tibasosa Der Entrada Contratistas,Asistencia,Entrada,08:00
4749,People,38613828,MARIANA SANCHEZ QUIROGA,Bavaria & CIA S.C.A,BP SUPPLY MANAGER,2025-06-12,14:11:00,Tibasosa Izq Salida Funcionarios,Asistencia,Salida,NaN
4750,People,38613828,MARIANA SANCHEZ QUIROGA,Bavaria & CIA S.C.A,BP SUPPLY MANAGER,2025-06-12,14:14:00,Tibasosa Der Entrada Contratistas,Asistencia,Entrada,16:00
